# 工具包导入&数据读取
## 工具包导入

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
import gc
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline 

/home/admin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%pwd

'/home/admin/Aliyun/Alibaba-3rd-Security-Algorithm-Challenge-master'

## 数据读取
- 为了方便分析，我们读取3000万条数据进行处理

In [3]:
# path = '../final_input/'
# train = pd.read_csv(path + 'final_train.csv',nrows=1000000)
# test = pd.read_csv(path + 'final_test.csv',nrows=1000000)

In [4]:

train = pd.read_csv('../train.csv')
#test = pd.read_csv(path + 'final_test.csv')

# 特征工程 & 验证结果(1-Gram)

In [5]:
train_data = train[['file_id','label']].drop_duplicates()
train_data.head()

,file_id,label
0,0,0
424,1,5
426,2,5
460,3,5
3260,4,5


In [6]:
train_data['label'].value_counts()

0    111545
5      3397
2       744
3       598
1       287
4        53
Name: label, dtype: int64

## 全局特征:
- File_id (Api): count,nunique
- File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Return Value): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

### File_id (Api): count,nunique

In [7]:
train.head()

,file_id,label,api,tid,return_value,index
0,0,0,GetSystemTimeAsFileTime,2644,0,0
1,0,0,NtAllocateVirtualMemory,2644,0,1
2,0,0,NtFreeVirtualMemory,2644,0,2
3,0,0,NtAllocateVirtualMemory,2644,0,3
4,0,0,NtAllocateVirtualMemory,2644,0,4


In [8]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api'].agg({'fileid_api_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')  

count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


In [9]:
train_data.head()

,file_id,label,fileid_api_count,fileid_api_nunique
0,0,0,424,19
1,1,5,2,2
2,2,5,34,15
3,3,5,2800,65
4,4,5,6832,78


### File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [10]:
tid_opt = ['count','nunique','max','min','median','std'] 
for opt in tid_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['tid'].agg({'fileid_tid_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique
max
min
median
std


In [11]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_tid_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['tid'].quantile(sec).values
 
train_data['fileid_tid_range'] = train.groupby(['file_id'])['tid'].quantile(0.975).values - train.groupby(['file_id'])['tid'].quantile(0.0125).values

### File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [12]:
index_opt = ['count','nunique','max','min','median','std'] 
for opt in index_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['index'].agg({'fileid_index_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique
max
min
median
std


In [13]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_index_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['index'].quantile(sec).values
 
train_data['fileid_index_range'] = train.groupby(['file_id'])['index'].quantile(0.975).values - train.groupby(['file_id'])['index'].quantile(0.0125).values

### 全局特征的线下验证 <font color=red>( 0.0969482)</font>

#### 评估指标

In [14]:
def lgb_logloss(preds,data):
    labels_ = data.get_label()
    classes_ = np.unique(labels_) 
    preds_prob = []
    
    for i in range(len(classes_)):
        preds_prob.append(preds[i*len(labels_):(i+1) * len(labels_)])
    #print(preds_prob)
    preds_prob_ = np.vstack(preds_prob) 
    
    loss = [] 
    for i in range(preds_prob_.shape[1]):  # 样本个数
        sum_ = 0  
        for j in range(preds_prob_.shape[0]): #类别个数
            pred = preds_prob_[j,i] # 第i个样本预测为第j类的概率
            if  j == labels_[i]:
                sum_ += np.log(pred)
            else:
                sum_ += np.log(1 - pred) 
             
        loss.append(sum_)  
         
    return 'loss is: ' ,-1 * (np.sum(loss) / preds_prob_.shape[1]),False

#### 训练特征 & 标签

In [15]:
train_features = [col for col in train_data.columns if col!='label' and col!='file_id']
train_label = 'label'

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split( train_data[train_features],train_data[train_label].values, test_size = 0.33) 
#del _
gc.collect()

train_ind = train_X.index
test_ind = test_X.index

In [29]:
(train_X.shape),type(train_Y)

((78138, 24), numpy.ndarray)

In [18]:
dtrain = lgb.Dataset(train_X,train_Y) 
dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

params = {
         'task':'train', 
         'num_leaves': 255,
         'objective': 'multiclass',
         'num_class':6,
        #'min_data_in_leaf': 40,
         'min_data_in_leaf': 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
         'bagging_freq': 5, 
         'max_bin':128,
        'num_threads': 10,
        'random_state':100
     }  
lgb_model_0_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50,feval=lgb_logloss)  

[1]	training's multi_logloss: 1.64895	training's loss is: : 2.52991	valid_1's multi_logloss: 1.65089	valid_1's loss is: : 2.53227
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 1.5265	training's loss is: : 2.37761	valid_1's multi_logloss: 1.53033	valid_1's loss is: : 2.38233
[3]	training's multi_logloss: 1.41946	training's loss is: : 2.24143	valid_1's multi_logloss: 1.42494	valid_1's loss is: : 2.24828
[4]	training's multi_logloss: 1.3244	training's loss is: : 2.11789	valid_1's multi_logloss: 1.33141	valid_1's loss is: : 2.12675
[5]	training's multi_logloss: 1.23913	training's loss is: : 2.00479	valid_1's multi_logloss: 1.24747	valid_1's loss is: : 2.01546
[6]	training's multi_logloss: 1.16219	training's loss is: : 1.90076	valid_1's multi_logloss: 1.17178	valid_1's loss is: : 1.91317
[7]	training's multi_logloss: 1.09203	training's loss is: : 1.80414	valid_1's multi_logloss: 1.10279	valid_1's loss is: : 1.81819
[8]	training's multi_logloss: 

[62]	training's multi_logloss: 0.0843635	training's loss is: : 0.164218	valid_1's multi_logloss: 0.128267	valid_1's loss is: : 0.23623
[63]	training's multi_logloss: 0.081106	training's loss is: : 0.158023	valid_1's multi_logloss: 0.125563	valid_1's loss is: : 0.231088
[64]	training's multi_logloss: 0.0780027	training's loss is: : 0.152087	valid_1's multi_logloss: 0.122951	valid_1's loss is: : 0.226128
[65]	training's multi_logloss: 0.0750579	training's loss is: : 0.146446	valid_1's multi_logloss: 0.120481	valid_1's loss is: : 0.22142
[66]	training's multi_logloss: 0.072228	training's loss is: : 0.141026	valid_1's multi_logloss: 0.11811	valid_1's loss is: : 0.216908
[67]	training's multi_logloss: 0.0695694	training's loss is: : 0.13592	valid_1's multi_logloss: 0.115906	valid_1's loss is: : 0.212705
[68]	training's multi_logloss: 0.0670089	training's loss is: : 0.130994	valid_1's multi_logloss: 0.113789	valid_1's loss is: : 0.208669
[69]	training's multi_logloss: 0.0646012	training's lo

[122]	training's multi_logloss: 0.0135009	training's loss is: : 0.026854	valid_1's multi_logloss: 0.0805337	valid_1's loss is: : 0.146537
[123]	training's multi_logloss: 0.0132081	training's loss is: : 0.0262747	valid_1's multi_logloss: 0.0805441	valid_1's loss is: : 0.146587
[124]	training's multi_logloss: 0.0129072	training's loss is: : 0.025679	valid_1's multi_logloss: 0.0805013	valid_1's loss is: : 0.146531
[125]	training's multi_logloss: 0.0126465	training's loss is: : 0.0251632	valid_1's multi_logloss: 0.0805682	valid_1's loss is: : 0.146685
[126]	training's multi_logloss: 0.0123659	training's loss is: : 0.0246081	valid_1's multi_logloss: 0.0805522	valid_1's loss is: : 0.14669
[127]	training's multi_logloss: 0.0120873	training's loss is: : 0.0240564	valid_1's multi_logloss: 0.0805353	valid_1's loss is: : 0.146703
[128]	training's multi_logloss: 0.01183	training's loss is: : 0.0235468	valid_1's multi_logloss: 0.0805219	valid_1's loss is: : 0.146723
[129]	training's multi_logloss: 

### 全局特征扩充
- File_id + return_value分段：计数

## 局部组合特征(展开形式)
### File_id + Api  
- File_id + Api (tid): count,nunique
- File_id + Api (return value): nunique, max, min, median, std
- File_id + Api (index):  nunique, max, min, median, std



#### File_id + Api (tid): count,nunique

In [19]:
def groupby_pivot_features(data_merge, data_orig , groupby_features,col1 = None, col2 = None, opts = None):
    for opt in opts:
        print(opt)
        train_split = data_orig.groupby(['file_id',col1])[col2].agg({'fileid_' + col1 + '_'+col2+'_'+ str(opt):opt}).reset_index() 
        
        train_split_ =  pd.pivot_table(train_split, values = 'fileid_' + col1 + '_'+col2+'_'+ str(opt), index=['file_id'],columns=[col1])
        new_cols = [ 'fileid_' + col1 + '_'+col2+  '_' + opt + '_' + str(col) for col in train_split_.columns]
        
        groupby_features.append(new_cols)
        train_split_.columns = new_cols 

        train_split_.reset_index(inplace = True)
        
        data_merge = pd.merge(data_merge,train_split_,how='left', on='file_id') 
    return data_merge,groupby_features 
    

In [20]:
groupby_features = []
api_opts = ['count', 'nunique']
train_data_,groupby_features = groupby_pivot_features(train_data, train, groupby_features, col1 = 'api', col2 = 'tid', opts = api_opts)

count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


#### File_id + Api (return value): nunique, max, min, median, std

In [21]:
# api_opts = ['nunique','max','min','median','std']
# train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'return_value', opts = api_opts) 

####  File_id + Api(index): nunique, max, min, median, std

In [22]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'index', opts = api_opts) 

nunique


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


max
min
median
std


In [27]:
train_data_.head()

,file_id,label,fileid_api_count,fileid_api_nunique,fileid_tid_count,fileid_tid_nunique,fileid_tid_max,fileid_tid_min,fileid_tid_median,fileid_tid_std,...,fileid_api_index_std_recv,fileid_api_index_std_recvfrom,fileid_api_index_std_select,fileid_api_index_std_send,fileid_api_index_std_sendto,fileid_api_index_std_setsockopt,fileid_api_index_std_shutdown,fileid_api_index_std_socket,fileid_api_index_std_system,fileid_api_index_std_timeGetTime
0,0,0,424,19,424,1,2644,2644,2644,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5,2,2,2,1,2524,2524,2524,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,34,15,34,1,2516,2516,2516,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5,2800,65,2800,5,2884,2508,2884,170.764080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.193663,NaN,NaN
4,4,5,6832,78,6832,6,2968,2060,2820,48.861741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1013.848858,NaN,NaN


In [28]:
len(train_data.columns)

26

### 1阶特征的线下验证(File_id + Api)（<font color=red>0.0347293</font>）

### File_id + Index  
- File_id + Index (api): count,nunique
- File_id + Index (return value): nunique, max, min, median, std(暂时先搁置)
- File_id + Index (tid):  nunique, max, min, median, std(暂时先搁置)


#### File_id +Tid (api): count,nunique

#### File_id + Index特征过拟合，删除


In [37]:
# delcol = []
# for i in range(2):
#     for item in groupby_features2[i]:
#         delcol.append(item)

In [38]:
# train_data_.drop(delcol,axis=1,inplace=True)

## 特征补充（加入index的差值特征）
- File_id + Api (index_diff): 'nunique','max','min','median','std'

In [25]:
train_diff = train.groupby(['file_id','tid'])['index'].diff().fillna(-999).values

In [26]:
train['index_diff'] = train_diff

In [27]:
train_diff = train.loc[train.index_diff!=-999] 

In [28]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train_diff, groupby_features, col1 = 'api', col2 = 'index_diff', opts = api_opts) 

nunique


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


max
min
median
std


In [29]:
train_data_.shape

(116624, 3718)

In [30]:
def lgb_logloss(preds,data):
    labels_ = data.get_label()
    classes_ = np.unique(labels_) 
    preds_prob = []
    for i in range(len(classes_)):
        preds_prob.append(preds[i*len(labels_):(i+1) * len(labels_)])
    preds_prob_ = np.vstack(preds_prob) 
    
    loss = [] 
    for i in range(preds_prob_.shape[1]):  # 样本个数
        sum_ = 0  
        for j in range(preds_prob_.shape[0]): #类别个数
            pred = preds_prob_[j,i] # 第i个样本预测为第j类的概率
            if  j == labels_[i]:
                sum_ += np.log(pred)
            else:
                sum_ += np.log(1 - pred) 
             
        loss.append(sum_)  
         
    return 'loss is: ' ,-1 * (np.sum(loss) / preds_prob_.shape[1]),False

### 线下验证(<font color=red>0.0346954</font>)

In [31]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
train_label = 'label'

#print(len(train_features))
#runXGB(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values,train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values)
dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

params = {
         'task':'train', 
         'num_leaves': 255,
         'objective': 'multiclass',
         'num_class':6,
         'min_data_in_leaf': 40,
         'learning_rate': 0.05,
         'feature_fraction': 0.85,
         'bagging_fraction': 0.9,
         'bagging_freq': 5, 
         'max_bin':128,
         'num_threads': 64,
         'random_state':100
     }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

[1]	training's multi_logloss: 1.64633	training's loss is: : 2.52671	valid_1's multi_logloss: 1.64655	valid_1's loss is: : 2.52698
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 1.52161	training's loss is: : 2.37151	valid_1's multi_logloss: 1.52201	valid_1's loss is: : 2.372
[3]	training's multi_logloss: 1.41271	training's loss is: : 2.23286	valid_1's multi_logloss: 1.41328	valid_1's loss is: : 2.23357
[4]	training's multi_logloss: 1.31623	training's loss is: : 2.10733	valid_1's multi_logloss: 1.31698	valid_1's loss is: : 2.10827
[5]	training's multi_logloss: 1.2298	training's loss is: : 1.99255	valid_1's multi_logloss: 1.23069	valid_1's loss is: : 1.99366
[6]	training's multi_logloss: 1.15164	training's loss is: : 1.88668	valid_1's multi_logloss: 1.15263	valid_1's loss is: : 1.88795
[7]	training's multi_logloss: 1.08064	training's loss is: : 1.78872	valid_1's multi_logloss: 1.08177	valid_1's loss is: : 1.79017
[8]	training's multi_logloss: 1

[62]	training's multi_logloss: 0.0687329	training's loss is: : 0.134376	valid_1's multi_logloss: 0.0762731	valid_1's loss is: : 0.146598
[63]	training's multi_logloss: 0.0657284	training's loss is: : 0.128563	valid_1's multi_logloss: 0.0733965	valid_1's loss is: : 0.141017
[64]	training's multi_logloss: 0.0628502	training's loss is: : 0.12299	valid_1's multi_logloss: 0.0706546	valid_1's loss is: : 0.135695
[65]	training's multi_logloss: 0.0601072	training's loss is: : 0.117673	valid_1's multi_logloss: 0.0680478	valid_1's loss is: : 0.130625
[66]	training's multi_logloss: 0.0575026	training's loss is: : 0.112621	valid_1's multi_logloss: 0.0655753	valid_1's loss is: : 0.125822
[67]	training's multi_logloss: 0.0550053	training's loss is: : 0.107776	valid_1's multi_logloss: 0.0632257	valid_1's loss is: : 0.121253
[68]	training's multi_logloss: 0.0526268	training's loss is: : 0.103156	valid_1's multi_logloss: 0.0609908	valid_1's loss is: : 0.116903
[69]	training's multi_logloss: 0.0503628	t

[122]	training's multi_logloss: 0.00554634	training's loss is: : 0.011025	valid_1's multi_logloss: 0.0206035	valid_1's loss is: : 0.0379255
[123]	training's multi_logloss: 0.00532893	training's loss is: : 0.0105945	valid_1's multi_logloss: 0.0204957	valid_1's loss is: : 0.0377147
[124]	training's multi_logloss: 0.00512331	training's loss is: : 0.0101872	valid_1's multi_logloss: 0.0204113	valid_1's loss is: : 0.0375447
[125]	training's multi_logloss: 0.00493127	training's loss is: : 0.00980648	valid_1's multi_logloss: 0.0203189	valid_1's loss is: : 0.0373628
[126]	training's multi_logloss: 0.00474015	training's loss is: : 0.00942797	valid_1's multi_logloss: 0.0202194	valid_1's loss is: : 0.0371783
[127]	training's multi_logloss: 0.0045591	training's loss is: : 0.00906936	valid_1's multi_logloss: 0.0201327	valid_1's loss is: : 0.0370164
[128]	training's multi_logloss: 0.00438636	training's loss is: : 0.00872696	valid_1's multi_logloss: 0.0200431	valid_1's loss is: : 0.0368479
[129]	train

[180]	training's multi_logloss: 0.000701126	training's loss is: : 0.00140006	valid_1's multi_logloss: 0.0210788	valid_1's loss is: : 0.0390168
[181]	training's multi_logloss: 0.000680197	training's loss is: : 0.00135831	valid_1's multi_logloss: 0.0211467	valid_1's loss is: : 0.0391498
[182]	training's multi_logloss: 0.000660359	training's loss is: : 0.00131873	valid_1's multi_logloss: 0.0212092	valid_1's loss is: : 0.0392737
[183]	training's multi_logloss: 0.000639805	training's loss is: : 0.00127772	valid_1's multi_logloss: 0.0212881	valid_1's loss is: : 0.0394297
[184]	training's multi_logloss: 0.000623048	training's loss is: : 0.00124428	valid_1's multi_logloss: 0.0213609	valid_1's loss is: : 0.0395699
[185]	training's multi_logloss: 0.000605285	training's loss is: : 0.00120884	valid_1's multi_logloss: 0.0214198	valid_1's loss is: : 0.0396854
[186]	training's multi_logloss: 0.000588288	training's loss is: : 0.00117492	valid_1's multi_logloss: 0.0214855	valid_1's loss is: : 0.0398139

In [26]:
len(train_features)

24

### 删除quantile,std统计变量之后的验证(<font color=red>0.0350054</font>)  

In [30]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [31]:
# train_data_.to_csv('/data/Data_JieZhang/TC_SAFE/train_val/train_data.csv',index = None) 

# 特征工程& 验证结果 2-Gram
## 全局特征
### File_id（Api_2）:count,nunique

In [30]:
train['api_shift'] = train['api'].shift(-1)
train['api_2'] = train['api'] +'_' + train['api_shift']

In [31]:
train.drop(['api_shift'],axis=1,inplace=True)

In [32]:
api_count = train['api_2'].value_counts()

In [ ]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api_2'].agg({'fileid_api_2_' + opt: opt}).reset_index() 
    train_data_ = pd.merge(train_data_,tmp,how='left', on='file_id')  

count


/home/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


## 局部特征
### File_id + tid (Api_2): count特征

In [ ]:
api_value_counts = pd.DataFrame(api_count).reset_index()
api_value_counts.columns = ['api_2','api_2_count']

In [ ]:
train = pd.merge(train, api_value_counts, on ='api_2' , how='left')

In [ ]:
api_opts = ['count']
groupby_features =  []
train_data_,groupby_features = groupby_pivot_features(train_data_, train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'tid', opts = api_opts)

### 线下验证(<font color=red> 0.0330886</font>)

In [ ]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
train_label = 'label'
print(len(train_features))
dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

params = {
         'task':'train', 
         'num_leaves': 255,
         'objective': 'multiclass',
         'num_class':6,
         'min_data_in_leaf': 40,
         'learning_rate': 0.05,
         'feature_fraction': 0.85,
         'bagging_fraction': 0.9,
         'bagging_freq': 5, 
         'max_bin':128,
         'num_threads': 64,
         'random_state':100
     }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### File_id + index (Api_2): max,min特征

In [ ]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
train_label = 'label'
print(len(train_features))
train_X, test_X, train_Y, test_Y = train_test_split( train_data_[train_features],train_data_[train_label].values, test_size = 0.33) 

dtrain = lgb.Dataset(train_X,train_Y) 
dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        #'min_data_in_leaf': 40,
        'min_data_in_leaf': 10,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [ ]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

In [ ]:
# train_data_[important_features].to_csv('../feature_final/train_data_2gram.csv',index = None)

train_ind = train_X.index
test_ind = test_X.index

In [ ]:
# len(important_features)

In [ ]:
api_opts = ['max','min']
train_data_,groupby_features = groupby_pivot_features(train_data_[important_features], train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'index', opts = api_opts)

In [ ]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
train_label = 'label'

train_ind = train_X.index
test_ind = test_X.index

dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        'min_data_in_leaf': 10,
        #'min_data_in_leaf': 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [ ]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

train_data_[important_features].to_csv('./train_data_2gram.csv',index = None)
 

##### 

# 附录
tf-idf的1Gram特征可以替换api的次数特征等，加入tf-idf有提升，提升较小